In [19]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from scipy.stats.mstats import winsorize

    1. ME에 CPI를 통해 inflation adjust 해주기 (CPI : Consumer Price Index)
    2. size, BM의 분포가 skew 되있으므로 log 취해줌
    3. correlation 구할 때 극단치 제거하기 위해 month별로 극단치를 winsorize해줌(0.5% 이하, 0.9%이상 값고정)
    4. 월별 groupby로 mean, std, skew등 통계치 구하고, variance별 groupby로 mean값 구함
    5. NYSE stock 기준으로 quantile 기준값을 구해서 size와 BM에 대해 5분위로 자르고 분류
    6. dependent sort(p1(size), p2(BM) 순서)하고 groupby (date, p1, p2 순서) 해서 permno를 count하고 전체 평균값을 구함

In [20]:
### sas 1 ### 
# daily market beta for US common stock 가져오기
# monthly_beta_data <- 용량이 커서 google colab 사용

# 경로설정
# 메모리에 한꺼번에 많은 걸 띄울 수 없어 일정 작업이후 저장 및 불러오기 반복
"""
import os, sys
from google.colab import drive
from dateutil.relativedelta import relativedelta
import pandas as pd

drive.mount('/content/gdrive')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)
cd /content/gdrive/My Drive/Colab Notebooks

assignment2_wrds = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/assignment2_wrds.csv')
assignment2_wrds['DATE'] = pd.to_datetime(assignment2_wrds['DATE'].map(lambda x : str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8]))
assignment2_wrds.to_csv('assignment2_wrds_tmp.csv')

assignment2_wrds_tmp = pd.read_csv('assignment2_wrds_tmp.csv', index_col = 'DATE')
assignment2_wrds_tmp.index = pd.to_datetime(assignment2_wrds_tmp.index)
# 메모리를 다 띄울 수 없어 두개로 나눔
# monthly beta로 바꿈
monthly_beta_data_1 = assignment2_wrds_tmp.iloc[:33638153,:].groupby('PERMNO').resample('1M').last()
monthly_beta_data_1.to_csv('monthly_beta_data_1.csv')
monthly_beta_data_2 = assignment2_wrds_tmp.iloc[33638153:,:].groupby('PERMNO').resample('1M').last()
monthly_beta_data_2.to_csv('monthly_beta_data_2.csv')

monthly_beta_data_1 = pd.read_csv('monthly_beta_data_1.csv')
monthly_beta_data_2 = pd.read_csv('monthly_beta_data_2.csv')

# 영업일 달말 데이터로 바꿔줌
monthly_beta_data_2 = monthly_beta_data_2.iloc[1:,:]
monthly_beta_data = pd.concat([monthly_beta_data_1, monthly_beta_data_2])
monthly_beta_data['DATE'] = pd.to_datetime(monthly_beta_data['DATE'])
monthly_beta_data['t'] = monthly_beta_data['DATE'].map(lambda x : x + relativedelta(months=1))
monthly_beta_data[['PERMNO','t', 'b_mkt']].to_csv('monthly_beta_data.csv')
"""

"\nimport os, sys\nfrom google.colab import drive\nfrom dateutil.relativedelta import relativedelta\nimport pandas as pd\n\ndrive.mount('/content/gdrive')\nnb_path = '/content/notebooks'\nos.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)\nsys.path.insert(0, nb_path)\ncd /content/gdrive/My Drive/Colab Notebooks\n\nassignment2_wrds = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/assignment2_wrds.csv')\nassignment2_wrds['DATE'] = pd.to_datetime(assignment2_wrds['DATE'].map(lambda x : str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8]))\nassignment2_wrds.to_csv('assignment2_wrds_tmp.csv')\n\nassignment2_wrds_tmp = pd.read_csv('assignment2_wrds_tmp.csv', index_col = 'DATE')\nassignment2_wrds_tmp.index = pd.to_datetime(assignment2_wrds_tmp.index)\n# 메모리를 다 띄울 수 없어 두개로 나눔\n# monthly beta로 바꿈\nmonthly_beta_data_1 = assignment2_wrds_tmp.iloc[:33638153,:].groupby('PERMNO').resample('1M').last()\nmonthly_beta_data_1.to_csv('monthly_beta_data_1.csv')\nmonthly_beta_data_2 = assig

In [21]:
# sas 2 
# monthly stock data에 beta data 추가
monthly_beta_data = pd.read_csv("monthly_beta_data.csv") #재석이 파일
monthly_beta_data["t"] = monthly_beta_data["t"].astype(str)
monthly_beta_data["t"] = pd.to_datetime(monthly_beta_data["t"])
monthly_beta_data["t"] = pd.to_datetime(monthly_beta_data["t"], format = '%Y%m')+ MonthEnd(0)
monthly_beta_data = monthly_beta_data[["PERMNO", "t", "b_mkt"]]
monthly_beta_data.columns = ['permno', 't', 'b_mkt'] #column 이름 바꿔줌
monthly_beta_data = monthly_beta_data[monthly_beta_data['t'] != pd.to_datetime('2013-01-31')]

assignment1_data = pd.read_csv(r".\assignment1_data.csv") #교수님 링크 파일
assignment1_data["t"] = assignment1_data["date"].astype(str)
assignment1_data["t"] = pd.to_datetime(assignment1_data["t"])
assignment1_data["t"] = pd.to_datetime(assignment1_data["t"], format = '%Y%m')+ MonthEnd(0)

monthly_stock_data_1 = assignment1_data.sort_values(by = ["permno", "t"]) #교수님 파일 column t calendar-day 마지막 날짜로 통일
monthly_stock_data_2 = pd.merge(monthly_stock_data_1, monthly_beta_data, how = 'left', left_on = ['permno', 't'], right_on = ['permno', 't'])
monthly_stock_data_2 = monthly_stock_data_2.loc[monthly_stock_data_2["b_mkt"].dropna().index]

In [22]:
# SAS 3
# Calculate CPI(Consumer Price Index)
# 6월의 CPI 계산
CPI_Jun = pd.read_excel('CPIAUCSL.xls', skiprows = 10 )
CPI_Jun["observation_date"] = CPI_Jun["observation_date"].astype(str)
CPI_Jun["observation_date"] = pd.to_datetime(CPI_Jun["observation_date"])
CPI_Jun = CPI_Jun[CPI_Jun["observation_date"].dt.month == 6]
CPI_Jun["t"] = CPI_Jun["observation_date"].dt.year
CPI_Jun = CPI_Jun[['t', 'CPIAUCSL']]
CPI_Jun.columns = ['t', 'cpi']
cpi_2012 = 228.524

# monthly_stock_data_2 <- month가 6보다 작은 애들은 year에서 1 빼주기
# 1~5월은 내년 return에 사용해야하기 때문
monthly_stock_data_2["date"] = monthly_stock_data_2["date"].astype(str)
monthly_stock_data_2["date"] = pd.to_datetime(monthly_stock_data_2["date"])
monthly_stock_data_2["date"] = pd.to_datetime(monthly_stock_data_2["date"], format = '%Y%m')+ MonthEnd(0)
monthly_stock_data_2["t"] = monthly_stock_data_2["t"].dt.year
first_filter = monthly_stock_data_2[monthly_stock_data_2["date"].dt.month <= 6].index
monthly_stock_data_2.loc[first_filter, "t"] = monthly_stock_data_2.loc[first_filter, "t"] - 1
monthly_stock_data_2 = monthly_stock_data_2.sort_values(by = ['t', 'date', 'permno'])

# Calculate Mktcap_CPI, Size, and log_BM variables

monthly_stock_data_3 = pd.merge(monthly_stock_data_2, CPI_Jun, left_on = 't', right_on = 't',  how = 'left')
monthly_stock_data_3["cpi"] = pd.to_numeric(monthly_stock_data_3["cpi"])
# inflation_adjust_mktcap = (mktcap * cpi[dec_2012]) / cpi
monthly_stock_data_3["ME_Jun_CPI"] = (monthly_stock_data_3["ME_Jun"]/monthly_stock_data_3["cpi"]) * cpi_2012
# distribution이 skew되있으므로 log를 취함
monthly_stock_data_3["size"] = np.log(monthly_stock_data_3["ME_Jun"])
monthly_stock_data_3["size_CPI"] = np.log(monthly_stock_data_3["ME_Jun_CPI"]) 
# BM distribution도 skew되있으므로 log를 취함
monthly_stock_data_3["log_BM"] = np.log(monthly_stock_data_3["BM"])
monthly_stock_data_3 = monthly_stock_data_3[["permno", "date", "year", "exchcd",
                                             "siccd", "retadj", "eretadj", "altprc_lag1",
                                             "ME_lag1", "b_mkt", "size", "size_CPI", "BM", "log_BM"]]

In [23]:
# SAS 4
# Calculate 0.5% and 99.5% level of each characteristic variable on a monthly basis;
monthly_stock_data_3 = monthly_stock_data_3.sort_values(by = 'date')
monthly_stock_data_4 = monthly_stock_data_3.copy()

# 바뀐 winsorization
winsorize_list = ['b_mkt', 'size', 'size_CPI', 'BM', 'log_BM']

# winsorize
for col in winsorize_list :
    monthly_stock_data_4[col] = monthly_stock_data_4[['date', col]].groupby('date').transform(lambda x : winsorize(x,limits=[0.005, 0.005]))

In [24]:
# sas 5
# Calculate summary statistics of variables in the "varlist" across stocks in each month,
# and stack the results in stats_by_year data set

monthly_stock_data_4 = monthly_stock_data_4.sort_values(by =['date', 'permno'])
stats_for_df = monthly_stock_data_4[['date', 'b_mkt', 'size', 'size_CPI', 'BM', 'log_BM']]

mean = pd.DataFrame(stats_for_df.groupby('date').mean().stack())
std = pd.DataFrame(stats_for_df.groupby('date').std().stack())[0].tolist()
skew = pd.DataFrame(stats_for_df.groupby('date').skew().stack())[0].tolist()
kurt = pd.DataFrame(stats_for_df.groupby('date').apply(pd.DataFrame.kurt).stack())[0].tolist()
min_ = pd.DataFrame(stats_for_df.groupby('date').min().stack())[0].tolist()
p25 = pd.DataFrame(stats_for_df.groupby('date').quantile([0.75]).stack())[0].tolist()
median = pd.DataFrame(stats_for_df.groupby('date').median().stack())[0].tolist()
p75 = pd.DataFrame(stats_for_df.groupby('date').quantile([0.25]).stack())[0].tolist()
max_ = pd.DataFrame(stats_for_df.groupby('date').max().stack())[0].tolist()
n = pd.DataFrame(stats_for_df.groupby('date').count().stack())[0].tolist()

mean['std'] = std
mean['skew'] = skew
mean['kurt'] = kurt
mean['min'] = min_
mean['p25'] = p25
mean['median'] = median
mean['p75'] = p75
mean['max'] = max_
mean['n'] = n

stats_by_month = mean.reset_index()
stats_by_month.columns = ['date', 'variables', 'mean', 'std', 'skew', 'kurt', 'min',
                          'p25', 'medain', 'p75', 'max', 'n']
stats_by_month = stats_by_month.sort_values(by = ['variables', 'date'])
stats_by_month = stats_by_month.groupby('variables').mean()
stats = stats_by_month.T[['b_mkt', 'size', 'size_CPI', 'BM', 'log_BM']].T

In [25]:
# sas 6
# Calculate the time-series-means of the correlations for variables in the "varlist"
pcorr_by_month = stats_for_df.groupby('date').corr()
pcorr_by_month = pcorr_by_month.reset_index()
pcorr_by_month.columns = ['date', 'variables', 'b_mkt', 'size', 'size_CPI', 'BM', 'log_BM']
pcorr_by_month = pcorr_by_month.groupby('variables').mean()
pcorr = pcorr_by_month.T[['b_mkt', 'size', 'size_CPI', 'BM', 'log_BM']].T

In [26]:
# sas 7
# Calculate size breakpoints
exchcd_filter = (monthly_stock_data_4["exchcd"] == 1) | (monthly_stock_data_4["exchcd"] == 31)
size_breakpoint = monthly_stock_data_4[exchcd_filter][['date', 'size']].groupby('date').quantile([0.2, 0.4, 0.6, 0.8])
size_breakpoint = size_breakpoint.unstack().T.reset_index().T.iloc[2:,:]
size_breakpoint.columns = ['20', '40', '60', '80']

monthly_stock_data_4_2 = monthly_stock_data_4.copy()
monthly_stock_data_4_2 = monthly_stock_data_4_2.set_index('date')
monthly_stock_data_5 = pd.concat([monthly_stock_data_4_2, size_breakpoint], axis =1)

In [27]:
# *** quanitle 20을 미리 만들어 두어야함
def classify(df, add_class) :
    # BM인 경우
    if add_class.split('_')[0] == 'BM' :
        # 1. 특정 quantile보다 작을 때 0을 부여
        df[add_class] = df['BM'] < df[add_class.split('_')[1] + '_bm']
        df[add_class] = df[add_class].map(lambda x : 0 if x == True else 1)

        # 2. 중복 제거 ( 앞선 quantile과 더해서 0이나온 부분만 합해줌, size_20에 우선 1을 붙여놓았기 때문임)
        if add_class == 'BM_40' : 
            df[add_class] = df['BM_20'] + df['BM_40'] == 0
        if add_class == 'BM_60' :
            df[add_class] = df['BM_20'] + df['BM_40'] + df['BM_60'] == 0
        if add_class == 'BM_80' :
            df[add_class] = df['BM_20'] + df['BM_40'] + df['BM_60'] + df['BM_80'] == 0

        # 3. 중복 제거된 부분만 1로 바꾸어줌
        df[add_class] = df[add_class].map(lambda x : 1 if x == True else 0)
    
    # Size인 경우
    else :
        # 1. 특정 quantile보다 작을 때 0을 부여
        df[add_class] = df['size'] < df[add_class.split('_')[1]]
        df[add_class] = df[add_class].map(lambda x : 0 if x == True else 1)

        # 2. 중복 제거 ( 앞선 quantile과 더해서 0이나온 부분만 합해줌, size_20에 우선 1을 붙여놓았기 때문임)
        if add_class == 'size_40' : 
            df[add_class] = df['size_20'] + df['size_40'] == 0
        if add_class == 'size_60' :
            df[add_class] = df['size_20'] + df['size_40'] + df['size_60'] == 0
        if add_class == 'size_80' :
            df[add_class] = df['size_20'] + df['size_40'] + df['size_60'] + df['size_80'] == 0

        # 3. 중복 제거된 부분만 1로 바꾸어줌
        df[add_class] = df[add_class].map(lambda x : 1 if x == True else 0)

In [28]:
def classify_specific(df, class_, new_p):
    df[new_p] = np.nan
    
    # quanitle 20을 미리 만들어 둠
    if class_ == 'BM' :
        df['{}_20'.format(class_)] = df['{}'.format(class_)] < df['20_bm']
    else :
        df['{}_20'.format(class_)] = df['{}'.format(class_)] < df['20']
    
    df['{}_20'.format(class_)] = df['{}_20'.format(class_)].map(lambda x : 1 if x== True else 0)

    classify(df, '{}_40'.format(class_))
    classify(df, '{}_60'.format(class_))
    classify(df, '{}_80'.format(class_))

    df['{}_40'.format(class_)] = df['{}_40'.format(class_)].map(lambda x : 2 if x == 1 else x)
    df['{}_60'.format(class_)] = df['{}_60'.format(class_)].map(lambda x : 3 if x == 1 else x)
    df['{}_80'.format(class_)] = df['{}_80'.format(class_)].map(lambda x : 4 if x == 1 else x)

    df[new_p] = df['{}_20'.format(class_)] + df['{}_40'.format(class_)] + df['{}_60'.format(class_)] + df['{}_80'.format(class_)]
    df[new_p] = df[new_p].map(lambda x: 5 if x == 0 else x)
    
    # 중간 과정 컬럼 제외하고 출력
    return df.iloc[:,:-4]

In [29]:
# size quantile 자르기
monthly_stock_data_5 = classify_specific(monthly_stock_data_5, 'size', 'p1')
monthly_stock_data_5 = monthly_stock_data_5.reset_index()
BM_breakpoint = monthly_stock_data_5[['date', 'p1', 'BM']].groupby(['date','p1']).quantile([0.2, 0.4, 0.6, 0.8])
BM_breakpoint = BM_breakpoint.unstack().T.reset_index().T.iloc[2:,:]
BM_breakpoint.columns = ['20_bm', '40_bm', '60_bm', '80_bm']

monthly_stock_data_5_2 = monthly_stock_data_5.copy()
monthly_stock_data_5_2 = monthly_stock_data_5_2.set_index('date')

BM_breakpoint = BM_breakpoint.reset_index()

BM_breakpoint = BM_breakpoint.set_index(['date','p1'])

monthly_stock_data_5_2 = monthly_stock_data_5.copy()
monthly_stock_data_5_2 = monthly_stock_data_5_2.set_index(['date','p1'])

monthly_stock_data_6 = pd.merge(monthly_stock_data_5_2, BM_breakpoint, how = 'left', left_on = ['date', 'p1'], right_on = ['date', 'p1'])
monthly_stock_data_6 = monthly_stock_data_6.reset_index()

# BM quantile 자르기
monthly_stock_data_6 = classify_specific(monthly_stock_data_6, 'BM', 'p2')

In [31]:
monthly_stock_data_6 = monthly_stock_data_6.sort_values(by = ['date', 'p1', 'p2'])

assignment2_data = monthly_stock_data_6.copy()

monthly_stock_data_7 = monthly_stock_data_6.copy()
nstocks_per_p = pd.DataFrame(monthly_stock_data_7.groupby(['date', 'p1', 'p2'])['permno'].count())
nstocks_per_p = nstocks_per_p.reset_index()
nstocks_per_p.columns = ['date', 'p1', 'p2', 'nstocks']
nstocks_per_p = pd.DataFrame(nstocks_per_p.groupby(['p1','p2'])['nstocks'].mean())

In [31]:
nstocks_per_p_final = nstocks_per_p.unstack()

In [48]:
ass_2_compare_sample = assignment2_data[assignment2_data['date'] == '2012-12-31'].sort_values(by = 'permno')

In [74]:
stat_list_str = ['monthly_stock_data_1','monthly_stock_data_2','CPI_Jun','monthly_stock_data_3','monthly_stock_data_4',
                 'stats','pcorr','size_breakpoint','monthly_stock_data_5',
                 'BM_breakpoint','monthly_stock_data_6','assignment2_data','nstocks_per_p_final']

### row 갯수 비교

In [75]:
stat_list = [monthly_stock_data_1,monthly_stock_data_2,CPI_Jun,monthly_stock_data_3,monthly_stock_data_4,
             stats,pcorr,size_breakpoint,monthly_stock_data_5,BM_breakpoint,monthly_stock_data_6,
             assignment2_data,nstocks_per_p_final]

for stat_str, stat in zip(stat_list_str, stat_list)  :
    print(stat_str), print(stat.shape)

monthly_stock_data_1
(1984019, 12)
monthly_stock_data_2
(1973918, 13)
CPI_Jun
(73, 2)
monthly_stock_data_3
(1973918, 14)
monthly_stock_data_4
(1973918, 14)
stats
(5, 10)
pcorr
(5, 5)
size_breakpoint
(599, 4)
monthly_stock_data_5
(1973918, 19)
BM_breakpoint
(2995, 4)
monthly_stock_data_6
(1973918, 24)
assignment2_data
(1973918, 24)
nstocks_per_p_final
(5, 5)


In [ ]:
"""
SAS 2
The data set WORK.DAILY_BETA_DATA has 67276307 observations and 11 variables.
The data set WORK.MONTHLY_BETA_DATA has 3225277 observations and 3 variables.
The data set WORK.MONTHLY_STOCK_DATA1 has 1984019 observations and 12 variables.
 The data set WORK.MONTHLY_STOCK_DATA2 has 1972906 observations and 13 variables.

SAS 3
The data set WORK.CPI_JUN has 73 observations and 2 variables.
The data set WORK.MONTHLY_STOCK_DATA2 has 1972906 observations and 13 variables.
The data set WORK.MONTHLY_STOCK_DATA3 has 1972906 observations and 14 variables.

SAS 4
The data set WORK.BOUNDS has 599 observations and 11 variables.
The data set WORK.MONTHLY_STOCK_DATA4 has 1972906 observations and 14 variables.

SAS 5
The data set WORK.STATS_BY_MONTH has 2995 observations and 13 variables.
The data set WORK.STATS has 5 observations and 12 variables (including "row_num").

SAS 6
The data set WORK.PCORR_BY_MONTH has 2995 observations and 8 variables.
The data set WORK.PCORR has 5 observations and 7 variables (including "row_num").

SAS 7
The data set WORK.SIZE_BREAKPOINTS has 599 observations and 5 variables.
The data set WORK.MONTHLY_STOCK_DATA5 has 1972906 observations and 19 variables.
The data set WORK.BM_BREAKPOINTS has 2995 observations and 6 variables.
The data set WORK.MONTHLY_STOCK_DATA6 has 1972906 observations and 24 variables.
The data set MY_LIB.ASSIGNMENT2_DATA has 1972906 observations and 24 variables.
The data set WORK.NSTOCKS_PER_P has 14975 observations and 6 variables.
The data set WORK.NSTOCKS_PER_P has 25 observations and 4 variables.
The data set WORK.NSTOCKS_PER_P has 5 observations and 6 variables.
"""

### DataFrame Table 출력

In [27]:
monthly_beta_data

,permno,t,b_mkt
0,10000,1986-11-30,0.7071
1,10000,1986-12-31,0.6398
2,10000,1987-01-31,0.6377
3,10000,1987-02-28,0.3729
4,10000,1987-03-31,0.3913
...,...,...,...
3285524,93436,2012-08-31,1.2045
3285525,93436,2012-09-30,1.2282
3285526,93436,2012-10-31,1.3440
3285527,93436,2012-11-30,1.4626


In [33]:
monthly_stock_data_1

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM,t
0,10001,19870731,1987,3,4920.0,0.021277,0.016677,5.875000,5.822125,5.822125,1.014415,1987-07-31
1,10001,19870831,1987,3,4920.0,0.083333,0.078633,6.000000,5.946000,5.822125,1.014415,1987-08-31
2,10001,19870930,1987,3,4920.0,-0.022308,-0.026808,6.500000,6.441500,5.822125,1.014415,1987-09-30
3,10001,19871030,1987,3,4920.0,0.020000,0.014000,6.250000,6.200000,5.822125,1.014415,1987-10-31
4,10001,19871130,1987,3,4920.0,-0.029412,-0.032912,6.375000,6.324000,5.822125,1.014415,1987-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...
1984014,93436,20120831,2012,3,9999.0,0.040117,0.040017,27.420000,2890.945448,3295.556766,0.074994,2012-08-31
1984015,93436,20120928,2012,3,9999.0,0.026648,0.026548,28.520000,3006.920688,3295.556766,0.074994,2012-09-30
1984016,93436,20121031,2012,3,9999.0,-0.039228,-0.039328,29.280001,3097.004233,3295.556766,0.074994,2012-10-31
1984017,93436,20121130,2012,3,9999.0,0.202215,0.202115,28.131399,3200.762464,3295.556766,0.074994,2012-11-30


In [34]:
monthly_stock_data_2

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM,t,b_mkt
10284,10137,1963-02-28,1963,1,4911.0,-0.004866,-0.007166,51.375000,479.739750,393.070500,0.443575,1962,1.0588
59565,10823,1963-02-28,1963,1,4925.0,-0.047771,-0.050071,39.250000,199.743250,173.662125,0.347801,1962,0.8783
97166,11404,1963-02-28,1963,1,4932.0,-0.024602,-0.026902,86.375000,1390.119250,1124.218875,0.630464,1962,0.7534
105703,11543,1963-02-28,1963,1,2011.0,-0.065789,-0.068089,9.500000,14.630000,10.929750,1.362077,1962,1.0872
115038,11674,1963-02-28,1963,1,4931.0,-0.075540,-0.077840,34.750000,998.228500,723.536125,0.531657,1962,0.4470
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983935,93428,2012-12-31,2012,3,9999.0,0.148593,0.148493,31.629999,880.421027,798.498921,0.163951,2012,1.5204
1983953,93429,2012-12-31,2012,3,9999.0,0.007672,0.007572,29.980000,2616.414520,2415.688987,0.112264,2012,0.4912
1983971,93433,2012-12-31,2012,3,9999.0,-0.517704,-0.517804,0.850100,39.250816,28.621680,1.187065,2012,1.8107
1983989,93434,2012-12-31,2012,3,9999.0,0.037634,0.037534,7.440000,58.575120,36.358170,0.569317,2012,0.3931


In [35]:
CPI_Jun

,t,cpi
5,1947,22.080
17,1948,24.150
29,1949,23.920
41,1950,23.880
53,1951,25.930
...,...,...
821,2015,237.684
833,2016,240.167
845,2017,244.182
857,2018,251.134


In [36]:
monthly_stock_data_3

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,b_mkt,size,size_CPI,BM,log_BM
0,10137,1963-02-28,1963,1,4911.0,-0.004866,-0.007166,51.375,479.739750,1.0588,5.973989,7.997457,0.443575,-0.812889
52,24360,1963-02-28,1963,1,4931.0,0.015945,0.013645,54.875,486.357125,1.2432,6.003609,8.027077,0.397120,-0.923518
51,24352,1963-02-28,1963,1,4911.0,-0.002268,-0.004568,55.125,118.022625,0.2664,4.516642,6.540110,0.504950,-0.683296
50,24301,1963-02-28,1963,1,4911.0,0.008000,0.005700,46.875,195.937500,1.0780,5.024193,7.047661,0.314440,-1.156963
49,24299,1963-02-28,1963,1,4931.0,-0.002736,-0.005036,53.000,52.788000,0.2980,3.754281,5.777749,0.586121,-0.534229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971783,75646,2012-12-31,2012,1,2834.0,-0.010997,-0.011097,92.750,28523.871250,0.7176,10.256538,10.256538,0.198390,-1.617523
1971784,75649,2012-12-31,2012,1,2673.0,0.126260,0.126160,18.850,605.462012,1.8021,6.147788,6.147788,0.657456,-0.419378
1971785,75654,2012-12-31,2012,3,3823.0,0.058071,0.057971,35.820,1538.863007,1.7110,7.213074,7.213074,0.365931,-1.005312
1971775,75573,2012-12-31,2012,1,5943.0,-0.023810,-0.023910,3.360,958.376130,2.2237,6.418463,6.418463,1.250995,0.223940


In [37]:
monthly_stock_data_4

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,b_mkt,size,size_CPI,BM,log_BM
0,10137,1963-02-28,1963,1,4911.0,-0.004866,-0.007166,51.375000,479.739750,1.0588,5.973989,7.997457,0.443575,-0.812889
1,10823,1963-02-28,1963,1,4925.0,-0.047771,-0.050071,39.250000,199.743250,0.8783,5.157112,7.180580,0.347801,-1.056125
2,11404,1963-02-28,1963,1,4932.0,-0.024602,-0.026902,86.375000,1390.119250,0.7534,7.024844,9.048312,0.630464,-0.461299
3,11543,1963-02-28,1963,1,2011.0,-0.065789,-0.068089,9.500000,14.630000,1.0872,2.391488,4.414957,1.362077,0.309011
4,11674,1963-02-28,1963,1,4931.0,-0.075540,-0.077840,34.750000,998.228500,0.4470,6.584150,8.607619,0.531657,-0.631756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973913,93428,2012-12-31,2012,3,9999.0,0.148593,0.148493,31.629999,880.421027,1.5204,6.682734,6.682734,0.163951,-1.808187
1973914,93429,2012-12-31,2012,3,9999.0,0.007672,0.007572,29.980000,2616.414520,0.4912,7.789740,7.789740,0.112264,-2.186904
1973915,93433,2012-12-31,2012,3,9999.0,-0.517704,-0.517804,0.850100,39.250816,1.8107,3.354164,3.354164,1.187065,0.171484
1973916,93434,2012-12-31,2012,3,9999.0,0.037634,0.037534,7.440000,58.575120,0.3931,3.593419,3.593419,0.569317,-0.563317


In [39]:
stats

,mean,std,skew,kurt,min,p25,medain,p75,max,n
variables,,,,,,,,,,
b_mkt,0.809207,0.586931,0.489397,0.341001,-0.567692,1.162164,0.745910,0.392213,2.609426,3295.355593
size,4.592077,1.911199,0.300253,-0.298130,0.521198,5.882777,4.454832,3.184371,9.942861,3295.355593
size_CPI,5.459570,1.911199,0.300253,-0.298130,1.388690,6.750270,5.322325,4.051863,10.810354,3295.355593
BM,0.885500,0.724514,2.213092,8.062700,0.043319,1.132170,0.714233,0.412820,4.961358,3295.355593
log_BM,-0.488367,0.827464,-0.569652,0.915970,-3.402942,0.061688,-0.400684,-0.958360,1.489849,3295.355593


In [41]:
pcorr

,b_mkt,size,size_CPI,BM,log_BM
variables,,,,,
b_mkt,1.000000,0.297960,0.297960,-0.193629,-0.220800
size,0.297960,1.000000,1.000000,-0.290869,-0.251849
size_CPI,0.297960,1.000000,1.000000,-0.290869,-0.251849
BM,-0.193629,-0.290869,-0.290869,1.000000,0.852691
log_BM,-0.220800,-0.251849,-0.251849,0.852691,1.000000


In [42]:
size_breakpoint

,20,40,60,80
date,,,,
1963-02-28,4.70129,5.17033,5.7488,6.30139
1963-03-31,4.70129,5.17033,5.7488,6.30139
1963-04-30,4.70129,5.17033,5.7488,6.30139
1963-05-31,4.70129,5.17033,5.7488,6.30139
1963-06-30,4.70129,5.17033,5.7488,6.30139
...,...,...,...,...
2012-08-31,6.25405,7.2625,7.92221,8.94935
2012-09-30,6.25802,7.26619,7.92254,8.94885
2012-10-31,6.25477,7.2646,7.92419,8.95126


In [43]:
monthly_stock_data_5

,date,permno,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,b_mkt,size,size_CPI,BM,log_BM,20,40,60,80,p1
0,1963-02-28,10137,1963,1,4911.0,-0.004866,-0.007166,51.375000,479.739750,1.0588,5.973989,7.997457,0.443575,-0.812889,4.70129,5.17033,5.7488,6.30139,4
1,1963-02-28,10823,1963,1,4925.0,-0.047771,-0.050071,39.250000,199.743250,0.8783,5.157112,7.180580,0.347801,-1.056125,4.70129,5.17033,5.7488,6.30139,2
2,1963-02-28,11404,1963,1,4932.0,-0.024602,-0.026902,86.375000,1390.119250,0.7534,7.024844,9.048312,0.630464,-0.461299,4.70129,5.17033,5.7488,6.30139,5
3,1963-02-28,11543,1963,1,2011.0,-0.065789,-0.068089,9.500000,14.630000,1.0872,2.391488,4.414957,1.362077,0.309011,4.70129,5.17033,5.7488,6.30139,1
4,1963-02-28,11674,1963,1,4931.0,-0.075540,-0.077840,34.750000,998.228500,0.4470,6.584150,8.607619,0.531657,-0.631756,4.70129,5.17033,5.7488,6.30139,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973913,2012-12-31,93428,2012,3,9999.0,0.148593,0.148493,31.629999,880.421027,1.5204,6.682734,6.682734,0.163951,-1.808187,6.25429,7.26262,7.92353,8.95458,2
1973914,2012-12-31,93429,2012,3,9999.0,0.007672,0.007572,29.980000,2616.414520,0.4912,7.789740,7.789740,0.112264,-2.186904,6.25429,7.26262,7.92353,8.95458,3
1973915,2012-12-31,93433,2012,3,9999.0,-0.517704,-0.517804,0.850100,39.250816,1.8107,3.354164,3.354164,1.187065,0.171484,6.25429,7.26262,7.92353,8.95458,1
1973916,2012-12-31,93434,2012,3,9999.0,0.037634,0.037534,7.440000,58.575120,0.3931,3.593419,3.593419,0.569317,-0.563317,6.25429,7.26262,7.92353,8.95458,1


In [44]:
BM_breakpoint

20_bm     40_bm     60_bm     80_bm
date       p1                                        
1963-02-28 1   0.438841  0.476999  0.506397    0.5742
           2   0.358932   0.40852  0.471159  0.563319
           3   0.309574   0.34893  0.447809  0.541939
           4   0.362776  0.401136  0.435341  0.481914
           5   0.373084   0.45651   0.48777  0.526183
...                 ...       ...       ...       ...
2012-12-31 1   0.424724    0.7581   1.08422   1.59828
           2   0.301001   0.47657  0.712597  0.969677
           3    0.31472  0.477534  0.684852  0.955739
           4   0.230689   0.39476  0.586259  0.894784
           5   0.229093  0.356126  0.551084  0.855467

[2995 rows x 4 columns]

In [45]:
monthly_stock_data_6

,date,p1,permno,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,...,log_BM,20,40,60,80,20_bm,40_bm,60_bm,80_bm,p2
5,1963-02-28,1,12781,1963,1,4925.0,-0.040000,-0.042300,31.250000,113.062500,...,-0.889445,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
17,1963-02-28,1,22437,1963,1,4931.0,0.003717,0.001417,33.625000,113.316250,...,-0.865339,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
72,1963-02-28,1,27385,1963,1,4931.0,-0.007353,-0.009653,51.000000,105.876000,...,-1.135734,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
23,1963-02-28,1,23042,1963,1,4911.0,0.008454,0.006154,48.500000,43.650000,...,-0.795035,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,2
41,1963-02-28,1,24117,1963,1,4931.0,-0.001263,-0.003563,47.500000,107.445000,...,-0.813454,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973087,2012-12-31,5,89525,2012,3,4841.0,0.008805,0.008705,37.195000,96040.388383,...,0.199701,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973106,2012-12-31,5,89626,2012,3,6200.0,-0.051565,-0.051665,55.270000,18426.133833,...,0.579987,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973372,2012-12-31,5,90880,2012,1,6282.0,0.032306,0.032206,60.669998,12486.856343,...,-0.090939,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973455,2012-12-31,5,91277,2012,3,4841.0,0.019953,0.019853,19.295000,12621.925545,...,0.066006,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5


In [46]:
assignment2_data

,date,p1,permno,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,...,log_BM,20,40,60,80,20_bm,40_bm,60_bm,80_bm,p2
5,1963-02-28,1,12781,1963,1,4925.0,-0.040000,-0.042300,31.250000,113.062500,...,-0.889445,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
17,1963-02-28,1,22437,1963,1,4931.0,0.003717,0.001417,33.625000,113.316250,...,-0.865339,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
72,1963-02-28,1,27385,1963,1,4931.0,-0.007353,-0.009653,51.000000,105.876000,...,-1.135734,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,1
23,1963-02-28,1,23042,1963,1,4911.0,0.008454,0.006154,48.500000,43.650000,...,-0.795035,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,2
41,1963-02-28,1,24117,1963,1,4931.0,-0.001263,-0.003563,47.500000,107.445000,...,-0.813454,4.70129,5.17033,5.7488,6.30139,0.438841,0.476999,0.506397,0.5742,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973087,2012-12-31,5,89525,2012,3,4841.0,0.008805,0.008705,37.195000,96040.388383,...,0.199701,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973106,2012-12-31,5,89626,2012,3,6200.0,-0.051565,-0.051665,55.270000,18426.133833,...,0.579987,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973372,2012-12-31,5,90880,2012,1,6282.0,0.032306,0.032206,60.669998,12486.856343,...,-0.090939,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5
1973455,2012-12-31,5,91277,2012,3,4841.0,0.019953,0.019853,19.295000,12621.925545,...,0.066006,6.25429,7.26262,7.92353,8.95458,0.229093,0.356126,0.551084,0.855467,5


In [30]:
ass_2_compare_sample

,date,p1,permno,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,...,log_BM,20,40,60,80,20_bm,40_bm,60_bm,80_bm,p2
1970712,2012-12-31,1,10001,2012,2,4925.0,-0.015231,-0.015331,9.520000,77.654644,...,-0.179014,6.25429,7.26262,7.92353,8.95458,0.424724,0.7581,1.08422,1.59828,3
1970713,2012-12-31,1,10002,2012,3,6020.0,0.010946,0.010846,2.749900,49.404705,...,1.079140,6.25429,7.26262,7.92353,8.95458,0.424724,0.7581,1.08422,1.59828,5
1970714,2012-12-31,1,10025,2012,3,3081.0,-0.020992,-0.021092,60.500000,334.625500,...,-0.899741,6.25429,7.26262,7.92353,8.95458,0.424724,0.7581,1.08422,1.59828,1
1970715,2012-12-31,2,10026,2012,3,2052.0,0.018173,0.018073,62.900002,1179.312129,...,-0.742029,6.25429,7.26262,7.92353,8.95458,0.301001,0.47657,0.712597,0.969677,2
1970716,2012-12-31,2,10032,2012,3,3670.0,0.114471,0.114371,23.150000,812.125137,...,-0.528508,6.25429,7.26262,7.92353,8.95458,0.301001,0.47657,0.712597,0.969677,3


In [79]:
pd.read_csv('assignment2_sample_data.csv').head()

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,b_mkt,...,size_20,size_40,size_60,size_80,p1,BM_20,BM_40,BM_60,BM_80,p2
0,10001,20121231,2012,2,4925,-0.015231,-0.015331,9.520000,77.654644,0.136698,...,6.253804,7.262689,7.922874,8.957895,1,0.424724,0.758100,1.084218,1.598276,3
1,10002,20121231,2012,3,6020,0.010946,0.010846,2.749900,49.404705,1.251151,...,6.253804,7.262689,7.922874,8.957895,1,0.424724,0.758100,1.084218,1.598276,5
2,10025,20121231,2012,3,3081,-0.020992,-0.021092,60.500000,334.625500,0.732402,...,6.253804,7.262689,7.922874,8.957895,1,0.424724,0.758100,1.084218,1.598276,1
3,10026,20121231,2012,3,2052,0.018173,0.018073,62.900002,1179.312129,0.959478,...,6.253804,7.262689,7.922874,8.957895,2,0.300471,0.476456,0.713487,0.973412,2
4,10032,20121231,2012,3,3670,0.114471,0.114371,23.150000,812.125137,1.747265,...,6.253804,7.262689,7.922874,8.957895,2,0.300471,0.476456,0.713487,0.973412,3


In [47]:
nstocks_per_p_final

nstocks                                                
p2           1           2           3           4           5
p1                                                            
1   391.595993  391.161937  391.178631  391.161937  391.787980
2    94.939900   94.517529   94.515860   94.517529   95.146912
3    67.158598   66.736227   66.739566   66.736227   67.363940
4    56.046745   55.697830   55.694491   55.697830   56.278798
5    50.347245   49.939900   49.918197   49.939900   50.535893